# Experiment 6

This experiment will explore agent performance in three pipe networks, Net3 (WNTR example - real network), 250701 K709vs2-Export (Sheffield example - real network), and Net6 (WNTR example and Paper one - real network).

Metrics that will be examined, percentage explored of nodes and links over 100 turns, absolute number of nodes and links explored over 100 turns, the node and links novelty scores over 100 turns and the stability of each control strategy with respect to starting positions of agents in the networks.

In [ ]:
## Imports

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import networkx as nx

import logging

from src.simulation import Simulation
from src.network import Network
from src.render import Render

logging.disable(logging.CRITICAL)

In [ ]:
network_file_1 = "networks/Environment1.inp"
network_file_2 = "networks/Environment2.inp"
network_file_3 = "networks/Environment3.inp"

env1 = Network(network_file_1)
env2 = Network(network_file_2)
env3 = Network(network_file_3)

g_env1 = env1.water_network_model.to_graph().to_undirected()
g_env2 = env2.water_network_model.to_graph().to_undirected()
g_env3 = env3.water_network_model.to_graph().to_undirected()

d_env1 = g_env1.degree
d_env2 = g_env2.degree
d_env3 = g_env3.degree

env1_start_pool = [node for node, degree in d_env1 if degree == 1]
env2_start_pool = [node for node, degree in d_env2 if degree == 1]
env3_start_pool = [node for node, degree in d_env3 if degree == 1]

env1_num_links = env1.graph_num_links
env2_num_links = env2.graph_num_links
env3_num_links = env3.graph_num_links

print("Start Pool - Environment1: ", env1_start_pool)
print("Number of Start Nodes - Environment1: ", len(env1_start_pool))
print("Number of Nodes - Environment1: ", env1.graph_num_nodes)
print("Number of Links - Environment1: ", env1.graph_num_links)

print("Start Pool - Environment2: ", env2_start_pool)
print("Number of Start Nodes - Environment2: ", len(env2_start_pool))
print("Number of Nodes - Environment2: ", env2.graph_num_nodes)
print("Number of Links - Environment2: ", env2.graph_num_links)

print("Start Pool - Environment3: ", env3_start_pool)
print("Number of Start Nodes - Environment3: ", len(env3_start_pool))
print("Number of Nodes - Environment3: ", env3.graph_num_nodes)
print("Number of Links - Environment3: ", env3.graph_num_links)

In [ ]:
def run_simulation_batch(env, num_agents, start_nodes, filepath, max_turns=100):
    print("Starting Simulation Batch - ", filepath)
    print("Number of Agents: ", num_agents)
    print("Start Nodes: ", start_nodes)
    print("Max Turns: ", max_turns)
    
    # Run the simulations for no swarm control
    path = f'{filepath}/NoSwarm'
    simulations_1 = []
    for node in start_nodes:
        print("Starting No Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=False, start_positions=[node], filepath=path)
        simulations_1.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_1
    
    # Run the simulations for naive swarm control
    path = f'{filepath}/NaiveSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'naive'}
    simulations_2 = []
    for node in start_nodes:
        print("Starting Naive Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_2.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_2
    
    # Run the simulations for informed mean swarm control
    path = f'{filepath}/InformedMeanSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'mean'}
    simulations_3 = []
    for node in start_nodes:
        print("Starting Informed Mean Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_3.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)    
    yield simulations_3
    
    # Run the simulations for informed median swarm control
    path = f'{filepath}/InformedMedianSwarm'
    swarm_config = {'swarm': True, 'swarm_type': 'informed', 'allocation_threshold': 'median'}
    simulations_4 = []
    for node in start_nodes:
        print("Starting Informed Median Swarm Simulation from Start Node: ", node)
        sim = Simulation(env, num_agents, swarm=True, swarm_config=swarm_config, start_positions=[node], filepath=path)
        simulations_4.append((node, sim.path_to_results_directory))
        sim.run(max_turns=max_turns)
    yield simulations_4

## Simulation 6.1

This set of simulations will focus on `env1`. There will be 10 agents, and the number of turns will be 100. In each simulation, all agents will begin at the same start node though to reduce the influence of the starting position on the final resuslts, the simulation will be run 15 times with the first 15 nodes in the starting node pool as the agent starting positions. The number of unique links explored will be recorded for each simulation.

In [ ]:
start_nodes = env1_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-6/Env1"

print("Start Nodes: ", start_nodes)

In [ ]:
simulations_6_1 = run_simulation_batch(env1, num_agents, start_nodes, filepath, max_turns=max_turns)

In [ ]:
no_swarm_env1 = next(simulations_6_1)

In [ ]:
naive_swarm_env1 = next(simulations_6_1)

In [ ]:
informed_mean_swarm_env1 = next(simulations_6_1)

In [ ]:
informed_median_swarm_env1 = next(simulations_6_1)

## Simulation 6.2

This set of simulations will focus on `env2`. There will be 10 agents, and the number of turns will be 100. In each simulation, all agents will begin at the same start node though to reduce the influence of the starting position on the final resuslts, the simulation will be run 15 times with the first 15 nodes in the starting node pool as the agent starting positions. The number of unique links explored will be recorded for each simulation.

In [ ]:
start_nodes = env2_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-6/Env2"

print("Start Nodes: ", start_nodes)

In [ ]:
simulations_6_2 = run_simulation_batch(env2, num_agents, start_nodes, filepath, max_turns=max_turns)

In [ ]:
no_swarm_env2 = next(simulations_6_2)

In [ ]:
naive_swarm_env2 = next(simulations_6_2)

In [ ]:
informed_mean_swarm_env2 = next(simulations_6_2)

In [ ]:
informed_median_swarm_env2 = next(simulations_6_2)

## Simulation 6.3

This set of simulations will focus on `env3`. There will be 10 agents, and the number of turns will be 100. In each simulation, all agents will begin at the same start node though to reduce the influence of the starting position on the final resuslts, the simulation will be run 15 times with the first 15 nodes in the starting node pool as the agent starting positions. The number of unique links explored will be recorded for each simulation.

In [ ]:
start_nodes = env3_start_pool[:15]
num_agents = 10
max_turns = 100
filepath = "notable-results/Experiment-6/Env3"

print("Start Nodes: ", start_nodes)

In [ ]:
simulations_6_3 = run_simulation_batch(env3, num_agents, start_nodes, filepath, max_turns=max_turns)

In [ ]:
no_swarm_env3 = next(simulations_6_3)

In [ ]:
naive_swarm_env3 = next(simulations_6_3)

In [ ]:
informed_mean_swarm_env3 = next(simulations_6_3)

In [ ]:
informed_median_swarm_env3 = next(simulations_6_3)

# Analysis

### Dataframe/Results functions

In [ ]:
from typing import List

# Read the results from the simulations
def read_results(simulations):
    results = []
    for sim in simulations:
        start_node, path = sim
        df = pd.read_csv(f"{path}/results.csv")
        df['start_node'] = start_node
        df.start_node = start_node
        results.append(df)
    return results
            
# Make the results into a single dataframe with turns as the columns and the data of interest as the rows
def make_results_dataframe(results, data_of_interest):
    df = pd.DataFrame()
    # Get the data of interest
    data = [result[['turn', data_of_interest, 'start_node']] for result in results]
    # Make the turn number the columns
    for result in data:
        start_node = result.start_node.unique()[0]

        result = result.T
        result.columns = result.iloc[0].astype(int)
        result = result.drop('turn')
        
        # Drop the start node row
        result = result.drop('start_node')
        # Add the start node to the dataframe
        result['start_node'] = start_node
        
        if df.empty:
            df = result
        else:
            df = pd.concat([df, result])
            
    # Check that each column that is a turn number has data of numeric type and if not, make it numeric (float)
    for column in df.columns:
        if column != 'start_node':
            if not pd.api.types.is_numeric_dtype(df[column]):
                df[column] = pd.to_numeric(df[column], errors='coerce')
            
    # Get the mean, min, max and standard deviation for each column
    df.loc['mean'] = df.mean(numeric_only=True)
    df.loc['min'] = df.min(numeric_only=True)
    df.loc['max'] = df.max(numeric_only=True)
    df.loc['std'] = df.std(numeric_only=True)
            
    return df

# Get the results from the simulations and make them into a dataframe
def get_results(simulations, data_of_interest):
    results = read_results(simulations)
    df = make_results_dataframe(results, data_of_interest)
    return df

# Get a list of dataframes with the results from the simulations for each environment
def l_environment_dataframe(swarm_types:list, simulations:list):
    experiment_dfs = []
    for simulation in simulations:
        df = read_results(simulation)
        experiment_dfs.append(df)
        
    return experiment_dfs

# Get a dataframe with the results from the simulations for each environment
def environment_dataframe(swarm_types:list, simulations:list, data_of_interest):
    experiment_dfs = []
    for simulation in simulations:
        df = get_results(simulation, data_of_interest)
        experiment_dfs.append(df)
    
    dataframe = pd.DataFrame()
    
    for swarm_type, df in zip(swarm_types, experiment_dfs):
        dataframe[f'{swarm_type}-mean'] = df.loc['mean']
        dataframe[f'{swarm_type}-min'] = df.loc['min']
        dataframe[f'{swarm_type}-max'] = df.loc['max']
        dataframe[f'{swarm_type}-std'] = df.loc['std']
        
    # Drop any columns that are all NaN
    dataframe = dataframe.dropna(axis=1, how='all')
    # Drop any rows that are all NaN
    dataframe = dataframe.dropna(axis=0, how='all')
    
    return dataframe

def start_node_results_mean(swarm_types, sims):
    results = pd.DataFrame()
    
    for swarm_type, sim in zip(swarm_types, sims):
        for start_node, simulation_path in sim:
            path = f'{simulation_path}/results.csv'
            df = pd.read_csv(path)
            df.start_node = start_node
            # Get the mean pct_links_explored for the entire simulation run
            mean_pct_links_explored = df.pct_links_explored.mean()
            # Get the mean link_novelty_score for the entire simulation run
            mean_link_novelty_score = df.link_novelty_score.mean()
            # Add the mean values to the results dataframe for the given swarm type and start node
            temp = pd.DataFrame({'swarm_type': swarm_type, 'start_node': start_node, 'mean_pct_links_explored': mean_pct_links_explored, 'mean_link_novelty_score': mean_link_novelty_score}, index=[0])
            results = pd.concat([results, temp], ignore_index=True)
            
    return results

def start_node_results_max(swarm_types, sims):
    results = pd.DataFrame()
    
    for swarm_type, sim in zip(swarm_types, sims):
        for start_node, simulation_path in sim:
            path = f'{simulation_path}/results.csv'
            df = pd.read_csv(path)
            df.start_node = start_node
            # get the max pct_links_explored for the entire simulation run
            max_pct_links_explored = df.pct_links_explored.max()
            # get the mean link_novelty_score for the entire simulation run
            mean_link_novelty_score = df.link_novelty_score.mean()
            # add the mean values to the results dataframe for the given swarm type and start node
            temp = pd.DataFrame({'swarm_type': swarm_type, 'start_node': start_node, 'max_pct_links_explored': max_pct_links_explored, 'mean_link_novelty_score': mean_link_novelty_score}, index=[0])
            results = pd.concat([results, temp], ignore_index=True)
            
    return results

### Plotting Functions

In [ ]:
# Function to plot the error bars according to the min and max values
def plot_errorbar_min_max(df, swarm_type, ax, color, label, errorevery):
    ax.errorbar(
        df.index,
        df[f'{swarm_type}-mean'],
        yerr=[df[f'{swarm_type}-mean'] - df[f'{swarm_type}-min'], df[f'{swarm_type}-max'] - df[f'{swarm_type}-mean']],
        errorevery=errorevery,
        label=label,
        color=color,
        capsize=5)

# Function to plot the error bars according to the standard deviation
def plot_errorbar_std(df, swarm_type, ax, color, label, errorevery):
    ax.errorbar(
        df.index,
        df[f'{swarm_type}-mean'],
        yerr=df[f'{swarm_type}-std'],
        errorevery=errorevery,
        color=color,
        capsize=5
    )
    
# Function to fill in the area between the error bars
def fill_area_between_min_and_max(dataframe, swarm_type, ax, colour='lightblue', alpha=0.3):
    x = dataframe.index.astype(int)
    y1 = dataframe[f'{swarm_type}-min'].astype(float)
    y2 = dataframe[f'{swarm_type}-max'].astype(float)
        
    ax.fill_between(x, y1, y2, color=colour, alpha=alpha)
 
swarm_type_to_linestyle = {
    'no-swarm': 'solid',
    'naive': 'dashed',
    'informed-mean': 'dotted',
    'informed-median': 'dashdot'
}
    
swarm_type_to_colour = {
    'no-swarm': 'black',
    'naive': 'red',
    'informed-mean': 'blue',
    'informed-median': 'green'
}

swarm_type_to_errorbar = {
    'no-swarm': (0,10),
    'naive': (3,10),
    'informed-mean': (6,10),
    'informed-median': (8,10)
}

# Plot the percentage of links explored by the swarm as a function of turns
def plot_pct_explored_results(results, swarm_types, title, ylabel, xlabel='Turns', figsize=(10,10)):
    # Set the figure size
    plt.figure(figsize=figsize)
    
    # Plot the mean and standard deviation for each swarm type
    for swarm_type in swarm_types:
        linestyle = swarm_type_to_linestyle[swarm_type]
        colour = swarm_type_to_colour[swarm_type]
        errorevery = swarm_type_to_errorbar[swarm_type]

        plt.plot(results[f'{swarm_type}-mean'], label=f'{swarm_type}-mean', color=colour, linestyle=linestyle)
            
        # fill_area_between_min_and_max(df, swarm_type, plt, colour=colour, alpha=0.15)
        
        
        plot_errorbar_std(results, swarm_type, plt, colour, f'{swarm_type}-std', errorevery)
        
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.legend()
    plt.show()
    
# Plot the link novelty as a function of turns
def plot_link_novelty(results, swarm_types, title, ylabel, xlabel='Turns', figsize=(10,10)):
    # Set the figure size
    plt.figure(figsize=figsize)
    
    # Plot the mean and standard deviation for each swarm type
    for swarm_type in swarm_types:
        linestyle = swarm_type_to_linestyle[swarm_type]
        colour = swarm_type_to_colour[swarm_type]
        errorevery = swarm_type_to_errorbar[swarm_type]

        plt.plot(results[f'{swarm_type}-mean'], label=f'{swarm_type}-mean', color=colour, linestyle=linestyle)
            
        # fill_area_between_min_and_max(df, swarm_type, plt, colour=colour, alpha=0.15)
        
        
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.legend()
    plt.show()

### Misc Functions/Variables

In [ ]:
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']

## Analysis 6.1.1

This analysis will focus on the results of the simulations in simulation 6.1. The results will be analysed to determine the effect of the swarm control strategy on the percentage of links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
results_6_1_1 = environment_dataframe(swarm_types, sims, 'pct_links_explored')
results_6_1_1

### Graphs

In [ ]:
# Plot the results for environment 1
plot_pct_explored_results(results_6_1_1, swarm_types, 'Percentage Links Explored per Turn - Environment 1', 'Percentage of Links Explored')


## Analysis 6.1.2

This analysis will focus on the results of the simulations in simulation 6.2. The results will be analysed to determine the effect of the swarm control strategy on the percentage of links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env2, naive_swarm_env2, informed_mean_swarm_env2, informed_median_swarm_env2]
results_6_1_2 = environment_dataframe(swarm_types, sims, 'pct_links_explored')
results_6_1_2

### Graphs

In [ ]:
# Plot the results for environment 2
plot_pct_explored_results(results_6_1_2, swarm_types, 'Percentage Links Explored per Turn - Environment 2', 'Percentage of Links Explored')

## Analysis 6.1.3

This analysis will focus on the results of the simulations in simulation 6.3. The results will be analysed to determine the effect of the swarm control strategy on the percentage of links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env3, naive_swarm_env3, informed_mean_swarm_env3, informed_median_swarm_env3]
results_6_1_3 = environment_dataframe(swarm_types, sims, 'pct_links_explored')
results_6_1_3

### Graphs

In [ ]:
# Plot the results for environment 3
plot_pct_explored_results(results_6_1_3, swarm_types, 'Percentage Links Explored per Turn - Environment 3', 'Percentage of Links Explored')

## Analysis 6.2.1

This analysis will focus on the results of the simulations in simulation 6.1. The results will be analysed to determine the effect of the swarm control strategy on the novelty of the links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
results_6_2_1 = environment_dataframe(swarm_types, sims, 'link_novelty_score')
results_6_2_1

### Graphs

In [ ]:
# Plot the results for environment 1
plot_link_novelty(results_6_2_1, swarm_types, 'Link Novelty Score per Turn - Environment 1', 'Link Novelty Score')

## Analysis 6.2.2

This analysis will focus on the results of the simulations in simulation 6.2. The results will be analysed to determine the effect of the swarm control strategy on the novelty of the links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env2, naive_swarm_env2, informed_mean_swarm_env2, informed_median_swarm_env2]
results_6_2_2 = environment_dataframe(swarm_types, sims, 'link_novelty_score')
results_6_2_2

### Graphs

In [ ]:
# Plot the results for environment 2
plot_link_novelty(results_6_2_2, swarm_types, 'Link Novelty Score per Turn - Environment 2', 'Link Novelty Score')

## Analysis 6.2.3

This analysis will focus on the results of the simulations in simulation 6.3. The results will be analysed to determine the effect of the swarm control strategy on the novelty of the links explored by the agents in the network at each turn.

### Results

In [ ]:
sims = [no_swarm_env3, naive_swarm_env3, informed_mean_swarm_env3, informed_median_swarm_env3]
results_6_2_3 = environment_dataframe(swarm_types, sims, 'link_novelty_score')
results_6_2_3

### Graphs

In [ ]:
# Plot the results for environment 3
plot_link_novelty(results_6_2_3, swarm_types, 'Link Novelty Score per Turn - Environment 3', 'Link Novelty Score')

## Analysis 6.3.1

This analysis will focus on the results of the simulations in simulation 6.1. The results will be analysed to determine the effect of the swarm control strategy on the stability of the control strategy with respect to the starting position of the agents in the network.

### Results

In [ ]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
results_6_3_1 = start_node_results_mean(swarm_types, sims)
results_6_3_1.sample(10)

In [ ]:
# Test if the correlation between start node and mean pct_links_explored is significant
results_6_3_1.groupby('start_node').mean_pct_links_explored.agg(['mean', 'std', 'count'])

### Graphs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='swarm_type', y='mean_pct_links_explored', data=results_6_3_1, ax=ax)
ax.set_title('Mean Percentage Links Explored per Swarm Type (all Start Nodes) - Environment 1')
ax.set_xlabel('Swarm Type')
ax.set_ylabel('Mean Percentage Links Explored (calculated across all turns)')
ax.grid(True, axis='y', alpha=0.5, linestyle='--')


## Analysis 6.3.2

This analysis will focus on the results of the simulations in simulation 6.2. The results will be analysed to determine the effect of the swarm control strategy on the stability of the control strategy with respect to the starting position of the agents in the network.

### Results

In [ ]:
sims = [no_swarm_env2, naive_swarm_env2, informed_mean_swarm_env2, informed_median_swarm_env2]
results_6_3_2 = start_node_results_mean(swarm_types, sims)
results_6_3_2.sample(10)

In [ ]:
# Test if the correlation between start node and mean pct_links_explored is significant
results_6_3_2.groupby('start_node').mean_pct_links_explored.agg(['mean', 'std', 'count'])

### Graphs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='swarm_type', y='mean_pct_links_explored', data=results_6_3_2, ax=ax)
ax.set_title('Mean Percentage Links Explored per Swarm Type (all Start Nodes) - Environment 2')
ax.set_xlabel('Swarm Type')
ax.set_ylabel('Mean Percentage Links Explored (calculated across all turns)')
ax.grid(True, axis='y', alpha=0.5, linestyle='--')

## Analysis 6.3.3

This analysis will focus on the results of the simulations in simulation 6.3. The results will be analysed to determine the effect of the swarm control strategy on the stability of the control strategy with respect to the starting position of the agents in the network.

### Results

In [ ]:
sims = [no_swarm_env3, naive_swarm_env3, informed_mean_swarm_env3, informed_median_swarm_env3]
results_6_3_3 = start_node_results_mean(swarm_types, sims)
results_6_3_3.sample(10)

In [ ]:
# Test if the correlation between start node and mean pct_links_explored is significant
results_6_3_3.groupby('start_node').mean_pct_links_explored.agg(['mean', 'std', 'count'])

### Graphs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='swarm_type', y='mean_pct_links_explored', data=results_6_3_3, ax=ax)
ax.set_title('Mean Percentage Links Explored per Swarm Type (all Start Nodes) - Environment 3')
ax.set_xlabel('Swarm Type')
ax.set_ylabel('Mean Percentage Links Explored (calculated across all turns)')
ax.grid(True, axis='y', alpha=0.5, linestyle='--')

In [ ]:
swarm_types = ['no-swarm', 'naive', 'informed-mean', 'informed-median']
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]

resultsq = pd.DataFrame()

for swarm_type, sim in zip(swarm_types, sims):
    for start_node, simulation_path in sim:
        path = f'{simulation_path}/results.csv'
        df = pd.read_csv(path)
        df.start_node = start_node
        # get the max pct_links_explored for the entire simulation run
        max_pct_links_explored = df.pct_links_explored.max()
        # get the mean link_novelty_score for the entire simulation run
        mean_link_novelty_score = df.link_novelty_score.mean()
        # add the mean values to the results dataframe for the given swarm type and start node
        temp = pd.DataFrame({'swarm_type': swarm_type, 'start_node': start_node, 'max_pct_links_explored': max_pct_links_explored, 'mean_link_novelty_score': mean_link_novelty_score}, index=[0])
        resultsq = pd.concat([resultsq, temp], ignore_index=True)
        
resultsq.sample(10)

In [ ]:
# get the mean max_pct_links_explored for each swarm type and the standard deviation
resultsq.groupby('swarm_type').max_pct_links_explored.agg(['mean', 'std', 'count'])

In [ ]:
# plot the mean max_pct_links_explored for each swarm type as a box plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='swarm_type', y='max_pct_links_explored', data=resultsq, ax=ax)

In [ ]:
sims = [no_swarm_env1, naive_swarm_env1, informed_mean_swarm_env1, informed_median_swarm_env1]
resultsz = start_node_results_max(swarm_types, sims)

# get the mean max_pct_links_explored for each swarm type and the standard deviation
resultsz.groupby('swarm_type').max_pct_links_explored.agg(['mean', 'std', 'count'])